# Importing libraries

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import f1_score
from sklearn.metrics import confusion_matrix
from sklearn.utils import resample
from imblearn.over_sampling import SMOTE
from IPython.display import Image

# Data Related with Covid

The lockdown in Spain was from the 8th of March to the 21 of June. We are going to analyse the influence to the creation of waste

In [11]:
df_waste19 = pd.read_excel("Modelo_Residuos_2019.xlsx", engine="openpyxl", sheet_name = "Hoja1")
df_waste20 = pd.read_excel("Modelo_Residuos_2020.xlsx", engine="openpyxl", sheet_name = "Hoja1")

In [12]:
df_waste19

,LOTE,DISTRITO,FRACCIÓN,ENERO '19(TN),FEBRERO '19(TN),MARZO '19(TN),ABRIL '19(TN),MAYO '19(TN),JUNIO'19(TN),JULIO'19(TN),AGOSTO'19(TN),SEPTIEMBRE'19(TN),OCTUBRE'19(TN),NOVIEMBRE'19(TN),DICIEMBRE'19(TN)
0,1,01. CENTRO,RESTO,6099.24,5643.16,6358.28,5923.36,6361.46,6244.32,5935.02,4864.96,5831.70,6375.50,6133.30,6593.32
1,1,01. CENTRO,ENVASES,201.66,191.92,219.30,205.92,225.40,222.86,205.70,177.78,225.74,238.30,229.12,238.16
2,1,01. CENTRO,VIDRIO,668.56,612.76,706.20,669.12,716.72,684.26,602.40,474.56,665.64,751.24,745.36,858.58
3,1,01. CENTRO,CLINICOS,139.10,152.88,173.58,170.48,165.50,160.88,157.64,122.64,153.26,182.52,171.12,154.98
4,1,01. CENTRO,ORGÁNICA,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
101,2,21. BARAJAS,RESTO,1446.82,1335.00,1460.46,1406.52,1544.66,1506.18,1463.34,1214.56,1454.38,1519.18,1435.88,1236.38
102,2,21. BARAJAS,ENVASES,170.62,142.20,176.78,160.08,152.74,162.48,112.28,127.06,170.32,189.40,167.06,201.14
103,2,21. BARAJAS,VIDRIO,145.38,128.40,136.42,124.80,133.40,123.36,125.30,116.26,128.92,125.18,134.62,149.42
104,2,21. BARAJAS,PUNTOS LIMPIOS,13.78,16.72,24.54,19.58,28.16,29.12,30.88,25.94,27.14,33.90,27.00,22.96


In [13]:
df_waste19['waste19'] = df_waste19.iloc[:,3:15].sum(axis=1)

In [14]:
df_waste19 = df_waste19[['DISTRITO', 'FRACCIÓN', 'waste19']]
df_waste19 = df_waste19.rename(columns={'DISTRITO':'district', 'FRACCIÓN':'waste_type'})
df_waste19.head()

,district,waste_type,waste19
0,01. CENTRO,RESTO,72363.62
1,01. CENTRO,ENVASES,2581.86
2,01. CENTRO,VIDRIO,8155.40
3,01. CENTRO,CLINICOS,1904.58
4,01. CENTRO,ORGÁNICA,0.00


In [15]:
df_waste19[['district_id', 'district_name']] = df_waste19['district'].str.split('.', expand=True)

In [16]:
df_waste19 = df_waste19.drop(['district'], axis= 1)

In [17]:
df_waste19 = df_waste19[['district_id', 'district_name', 'waste_type', 'waste19']]

In [19]:
def items_lower(df):
    df_cat = df.select_dtypes(['object'])
    for i in df_cat.columns:
        df[i] = list(map(lambda x: x.lower(), df[i]))
    return df

In [20]:
def accent_replace(df):
    df_cat = df.select_dtypes(['object'])
    for i in df_cat.columns:
        df[i]=df[i].str.replace('á', 'a')
        df[i]=df[i].str.replace('é', 'e')
        df[i]=df[i].str.replace('í', 'i')
        df[i]=df[i].str.replace('ó', 'o')
        df[i]=df[i].str.replace('ú', 'u')


    return df

In [21]:
df_waste19 = items_lower(df_waste19)
df_waste19 = accent_replace(df_waste19)

In [22]:
df_waste19['district_id'] = list(map(lambda x: int(x), df_waste19['district_id']))

In [23]:
df_waste19

,district_id,district_name,waste_type,waste19
0,1,centro,resto,72363.62
1,1,centro,envases,2581.86
2,1,centro,vidrio,8155.40
3,1,centro,clinicos,1904.58
4,1,centro,organica,0.00
...,...,...,...,...
101,21,barajas,resto,17023.36
102,21,barajas,envases,1932.16
103,21,barajas,vidrio,1571.46
104,21,barajas,puntos limpios,299.72


In [24]:
df_waste19 = df_waste19.groupby(['district_name']).agg({'district_id':np.mean, 'waste19': sum}).reset_index()

In [25]:
df_waste19['district_id'] = list(map(lambda x: int(x), df_waste19['district_id']))

In [26]:
df_waste19

,district_name,district_id,waste19
0,arganzuela,2,42969.52
1,barajas,21,21018.00
2,carabanchel,11,75868.74
3,centro,1,85005.46
4,chamartin,5,48791.14
5,chamberi,7,46281.22
6,ciudad lineal,15,62649.40
7,fuencarral,8,78917.38
8,hortaleza,16,58331.32
9,latina,10,67965.80


Do the same with the data in 2020, where lockdown happend

In [27]:
df_waste20['waste20'] = df_waste20.iloc[:,5:9].sum(axis=1)

In [28]:
df_waste20 = df_waste20[['DISTRITO', 'FRACCIÓN', 'waste20']]
df_waste20 = df_waste20.rename(columns={'DISTRITO':'district', 'FRACCIÓN':'waste_type'})
df_waste20.head()

,district,waste_type,waste20
0,01. CENTRO,RESTO,13118.18
1,01. CENTRO,ENVASES,813.80
2,01. CENTRO,VIDRIO,1446.04
3,01. CENTRO,CLINICOS,2014.94
4,01. CENTRO,ORGÁNICA,0.00


In [29]:
df_waste20[['district_id', 'district_name']] = df_waste20['district'].str.split('.', expand=True)

In [30]:
df_waste20 = df_waste20.drop(['district'], axis= 1)

In [32]:
df_waste20 = df_waste20[['district_id', 'district_name', 'waste_type', 'waste20']]

In [33]:
df_waste20 = items_lower(df_waste20)
df_waste20 = accent_replace(df_waste20)

In [34]:
df_waste20['district_id'] = list(map(lambda x: int(x), df_waste20['district_id']))

In [35]:
df_waste20 = df_waste20.groupby(['district_name']).agg({'district_id':np.mean, 'waste20': sum}).reset_index()

In [36]:
df_waste20['district_id'] = list(map(lambda x: int(x), df_waste20['district_id']))

In [37]:
df_waste20 = df_waste20.drop(['district_name', 'district_id'], axis=1)

In [38]:
df_covid = pd.concat([df_waste19, df_waste20], axis=1)

In [39]:
df_covid = df_covid.sort_values(by=['district_id'], ignore_index=True)

In [40]:
df_covid

,district_name,district_id,waste19,waste20
0,centro,1,85005.46,17392.96
1,arganzuela,2,42969.52,12551.10
2,retiro,3,36879.44,10785.22
3,salamanca,4,53533.76,13762.00
4,chamartin,5,48791.14,13398.38
5,tetuan,6,52020.12,14826.44
6,chamberi,7,46281.22,12412.62
7,fuencarral,8,78917.38,24684.84
8,moncloa - aravaca,9,56453.36,14905.38
9,latina,10,67965.80,22307.40


Residents to calculate the covid impact

In [41]:
df_res19 = pd.read_excel("Población_19.xlsx", engine="openpyxl", sheet_name = "MadridDatos 1.0")
df_res20 = pd.read_excel("Población_20.xlsx", engine="openpyxl", sheet_name = "MadridDatos 1.0")

In [42]:
df_res19[['district_id1', 'district_name1']] = df_res19['Distrito'].str.split('.', expand=True)
df_res20[['district_id2', 'district_name2']] = df_res20['Distrito'].str.split('.', expand=True)

In [43]:
df_res19 = df_res19.drop(['Distrito'], axis= 1)
df_res20 = df_res20.drop(['Distrito'], axis= 1)
df_res19 = df_res19.rename(columns={'Población':'residents19'})
df_res20 = df_res20.rename(columns={'Población':'residents20'})

In [44]:
df_res19 = df_res19[['district_id1', 'district_name1', 'residents19']]
df_res20 = df_res20[['district_id2', 'district_name2', 'residents20']]

In [45]:
df_res19['district_id1'] = list(map(lambda x: int(x), df_res19['district_id1']))
df_res20['district_id2'] = list(map(lambda x: int(x), df_res20['district_id2']))

In [46]:
df_res19 = items_lower(df_res19)
df_res20 = accent_replace(df_res20)

Joining the tables

In [47]:
df_covid = pd.concat([df_covid, df_res19, df_res20], axis=1)

In [48]:
df_covid = df_covid.drop(['district_id1', 'district_id2', 'district_name1', 'district_name2'], axis=1)

In [49]:
df_covid

,district_name,district_id,waste19,waste20,residents19,residents20
0,centro,1,85005.46,17392.96,134881,140473
1,arganzuela,2,42969.52,12551.10,153830,155660
2,retiro,3,36879.44,10785.22,119379,120406
3,salamanca,4,53533.76,13762.00,146148,147854
4,chamartin,5,48791.14,13398.38,145865,147551
5,tetuan,6,52020.12,14826.44,157937,161313
6,chamberi,7,46281.22,12412.62,139448,140866
7,fuencarral,8,78917.38,24684.84,246021,249973
8,moncloa - aravaca,9,56453.36,14905.38,119423,121683
9,latina,10,67965.80,22307.40,238154,242139


# Insights

Mean Variation during covid lockdown

In [50]:
df_covid['waste20'].sum()/df_covid['waste19'].sum()-1

-0.7006695582783484

In [53]:
df_covid['waste20'].sum()

335074.28

In [55]:
(df_covid['waste19'].sum() - df_covid['waste20'].sum())/225

3485.9482666666663

In [581]:
print(df_covid['waste19_months_covid'].sum()/df_covid['residents19'].sum()*1000)
print(df_covid['waste20_months_covid'].sum()/df_covid['residents20'].sum()*1000)


117.1271898267244
100.48018280340538
